# Chapter 13: Generics – Code Reusability with Type Safety

Imagine you want to create a collection class that can hold any type of object. Before generics, you might have used `ArrayList` from the `System.Collections` namespace, which stores `object` references. But then you lose type safety – you could accidentally put a `string` into a list of integers, and you'd need to cast every time you retrieve an item. Casting is error‑prone and hurts performance.

Generics solve this problem by allowing you to define classes, interfaces, and methods with a **placeholder type** that is specified when the code is used. This gives you the best of both worlds: code reuse across different types, and compile‑time type safety.

In this chapter, you'll learn:

- The benefits of generics.
- How to create **generic classes** and **generic methods**.
- Using **generic constraints** to restrict the types that can be used.
- The **`default` keyword** for generic types.
- **Covariance and contravariance** – how generic type parameters can be used with inheritance (`out` and `in`).
- Real‑world examples and best practices.

By the end, you'll be able to write flexible, reusable components that are also type‑safe and performant.

---

## 13.1 Why Generics?

Consider a simple stack implementation without generics:

```csharp
public class ObjectStack
{
    private object[] _items;
    private int _top;

    public ObjectStack(int capacity)
    {
        _items = new object[capacity];
        _top = -1;
    }

    public void Push(object item) => _items[++_top] = item;
    public object Pop() => _items[_top--];
}
```

You can use it like this:

```csharp
ObjectStack stack = new ObjectStack(10);
stack.Push(42);
stack.Push("Hello"); // Accidentally allowed – no type safety
int number = (int)stack.Pop(); // Cast required, might throw if type mismatch
```

Problems:
- No type safety – you can mix types.
- Performance overhead from boxing/unboxing for value types.
- Requires casting, which can fail at runtime.

Generics solve these issues:

```csharp
public class Stack<T>
{
    private T[] _items;
    private int _top;

    public Stack(int capacity)
    {
        _items = new T[capacity];
        _top = -1;
    }

    public void Push(T item) => _items[++_top] = item;
    public T Pop() => _items[_top--];
}

// Usage
Stack<int> intStack = new Stack<int>(10);
intStack.Push(42);
intStack.Push(100);
int value = intStack.Pop(); // No cast, type is int

// Stack<string> stringStack = new Stack<string>(5);
```

Now the compiler enforces that only integers go into `intStack`. No casting, no runtime surprises, and value types are stored without boxing.

---

## 13.2 Generic Classes

A generic class is defined with a **type parameter** in angle brackets after the class name. You can use the type parameter throughout the class.

```csharp
public class Pair<TFirst, TSecond>
{
    public TFirst First { get; set; }
    public TSecond Second { get; set; }

    public Pair(TFirst first, TSecond second)
    {
        First = first;
        Second = second;
    }

    public void Display()
    {
        Console.WriteLine($"First: {First}, Second: {Second}");
    }
}

// Usage
Pair<int, string> pair = new Pair<int, string>(42, "Answer");
pair.Display();
```

You can have multiple type parameters. They are typically named `T`, `TKey`, `TValue`, etc., but any identifier is allowed.

### Default Values for Generic Types

Sometimes you need to assign a default value to a variable of type `T`. You can't use `null` because `T` might be a value type. The `default` keyword returns the default value for the type (e.g., `0` for numbers, `null` for reference types, `false` for bool).

```csharp
public T GetDefault() => default(T); // or simply default
```

In a generic class, you might initialize an array with default values:

```csharp
public class Buffer<T>
{
    private T[] _buffer;

    public Buffer(int size)
    {
        _buffer = new T[size]; // all elements are default(T)
    }
}
```

---

## 13.3 Generic Methods

Methods can be generic even if the containing class is not. You define the type parameter right before the method's parameter list.

```csharp
public class Utilities
{
    public static T Max<T>(T a, T b) where T : IComparable<T>
    {
        return a.CompareTo(b) > 0 ? a : b;
    }
}

// Usage
int maxInt = Utilities.Max(5, 10); // type inference – T is int
string maxString = Utilities.Max("apple", "banana");
```

The compiler often infers the type parameter from the arguments, so you can omit `<int>`.

### Generic Methods in Non‑Generic Classes

```csharp
public class ArrayHelper
{
    public static void Swap<T>(ref T a, ref T b)
    {
        T temp = a;
        a = b;
        b = temp;
    }
}

int x = 5, y = 10;
ArrayHelper.Swap(ref x, ref y);
```

---

## 13.4 Generic Constraints

By default, a type parameter can be any type. Sometimes you need to restrict the types that can be used, e.g., to ensure they implement an interface or have a parameterless constructor. This is done with **constraints** using the `where` keyword.

### Common Constraints

| Constraint | Description |
|------------|-------------|
| `where T : struct` | T must be a value type. |
| `where T : class` | T must be a reference type. |
| `where T : notnull` | T must be a non‑nullable type (C# 8+). |
| `where T : new()` | T must have a public parameterless constructor. |
| `where T : <base class>` | T must derive from the specified base class. |
| `where T : <interface>` | T must implement the specified interface. |
| `where T : U` | T must derive from another type parameter U. |

You can combine multiple constraints.

#### Example: Requiring a Parameterless Constructor

```csharp
public class Factory<T> where T : new()
{
    public T CreateInstance()
    {
        return new T(); // allowed because of new() constraint
    }
}

class Person
{
    public Person() { } // parameterless constructor
}

var factory = new Factory<Person>();
Person p = factory.CreateInstance();
```

#### Example: Requiring an Interface

```csharp
public class Sorter<T> where T : IComparable<T>
{
    public void BubbleSort(T[] array)
    {
        for (int i = 0; i < array.Length - 1; i++)
            for (int j = 0; j < array.Length - i - 1; j++)
                if (array[j].CompareTo(array[j + 1]) > 0)
                {
                    T temp = array[j];
                    array[j] = array[j + 1];
                    array[j + 1] = temp;
                }
    }
}
```

Now you can sort any array of types that implement `IComparable<T>`.

#### Multiple Constraints

```csharp
public class Example<T> where T : class, IDisposable, new()
{
    // T must be a reference type, implement IDisposable, and have a parameterless constructor
}
```

Constraints are essential for writing generic code that can safely call methods or constructors on the type parameters.

---

## 13.5 Covariance and Contravariance

Covariance and contravariance deal with how generic types can be used with inheritance. They apply to interfaces and delegates that have variant type parameters marked with `out` (covariant) or `in` (contravariant).

### Covariance (`out`)

Covariance allows you to use a more derived type than originally specified. You can assign an `IEnumerable<Derived>` to an `IEnumerable<Base>`.

The `out` keyword means the type parameter is used only as a return type, not as a method parameter.

```csharp
IEnumerable<string> strings = new List<string> { "a", "b" };
IEnumerable<object> objects = strings; // Covariant conversion
```

Here, `IEnumerable<T>` is covariant because `T` is marked `out` in the interface definition.

**Creating a covariant interface:**

```csharp
public interface IProducer<out T>
{
    T Produce();
}

public class Producer<T> : IProducer<T>
{
    public T Produce() => default(T);
}

IProducer<string> stringProd = new Producer<string>();
IProducer<object> objectProd = stringProd; // OK – covariance
```

### Contravariance (`in`)

Contravariance allows you to use a less derived type than originally specified. You can pass an `Action<Base>` where an `Action<Derived>` is expected.

The `in` keyword means the type parameter is used only as a method parameter, not as a return type.

```csharp
Action<object> objectAction = obj => Console.WriteLine(obj);
Action<string> stringAction = objectAction; // Contravariant conversion
stringAction("Hello");
```

**Creating a contravariant interface:**

```csharp
public interface IConsumer<in T>
{
    void Consume(T item);
}

public class Consumer<T> : IConsumer<T>
{
    public void Consume(T item) => Console.WriteLine(item);
}

IConsumer<object> objectConsumer = new Consumer<object>();
IConsumer<string> stringConsumer = objectConsumer; // OK – contravariance
stringConsumer.Consume("Hello");
```

### Why Variance Matters

Variance makes generic interfaces and delegates more flexible. You'll encounter it frequently with built‑in types like `IEnumerable<T>` (covariant), `IComparer<T>` (contravariant), and `Func`/`Action` delegates.

- `Func<out TResult>` is covariant in the return type.
- `Action<in T>` is contravariant in the parameter type.

**Important:** Variance is not supported for generic classes, only for interfaces and delegates. Also, variant type parameters must be reference types (for historical reasons, value types don't support variance because they don't have inheritance).

---

## 13.6 Putting It All Together: A Generic Repository Example

Let's build a simple generic repository that can work with any entity type, demonstrating generic classes, constraints, and methods.

```csharp
using System;
using System.Collections.Generic;
using System.Linq;

namespace GenericRepositoryDemo
{
    // Base entity class with an Id
    public abstract class Entity
    {
        public int Id { get; set; }
    }

    // Generic repository interface
    public interface IRepository<T> where T : Entity
    {
        T GetById(int id);
        IEnumerable<T> GetAll();
        void Add(T entity);
        void Update(T entity);
        void Delete(int id);
    }

    // In‑memory implementation
    public class InMemoryRepository<T> : IRepository<T> where T : Entity
    {
        private readonly List<T> _entities = new List<T>();
        private int _nextId = 1;

        public T GetById(int id)
        {
            return _entities.FirstOrDefault(e => e.Id == id);
        }

        public IEnumerable<T> GetAll()
        {
            return _entities.ToList(); // return a copy
        }

        public void Add(T entity)
        {
            if (entity == null) throw new ArgumentNullException(nameof(entity));
            entity.Id = _nextId++;
            _entities.Add(entity);
        }

        public void Update(T entity)
        {
            if (entity == null) throw new ArgumentNullException(nameof(entity));
            var existing = GetById(entity.Id);
            if (existing == null)
                throw new InvalidOperationException("Entity not found");
            // In a real implementation, you'd update properties
            // Here we remove and add (simplified)
            _entities.Remove(existing);
            _entities.Add(entity);
        }

        public void Delete(int id)
        {
            var entity = GetById(id);
            if (entity != null)
                _entities.Remove(entity);
        }
    }

    // Example entity
    public class Person : Entity
    {
        public string Name { get; set; }
        public int Age { get; set; }
    }

    class Program
    {
        static void Main()
        {
            IRepository<Person> repository = new InMemoryRepository<Person>();

            // Add people
            repository.Add(new Person { Name = "Alice", Age = 30 });
            repository.Add(new Person { Name = "Bob", Age = 25 });

            // Get all
            Console.WriteLine("All people:");
            foreach (var p in repository.GetAll())
            {
                Console.WriteLine($"  {p.Id}: {p.Name}, {p.Age}");
            }

            // Get by ID
            var alice = repository.GetById(1);
            Console.WriteLine($"\nGot by ID 1: {alice?.Name}");

            // Update
            if (alice != null)
            {
                alice.Age = 31;
                repository.Update(alice);
            }

            // Delete
            repository.Delete(2); // Bob

            Console.WriteLine("\nAfter update and delete:");
            foreach (var p in repository.GetAll())
            {
                Console.WriteLine($"  {p.Id}: {p.Name}, {p.Age}");
            }

            // Demonstrate generic method with constraint
            Console.WriteLine("\nFind by name (using custom method):");
            var found = FindByName(repository, "Alice");
            if (found != null)
                Console.WriteLine($"Found: {found.Name}");
        }

        // Generic method with constraint (works with any IRepository<T> where T : Entity)
        static T FindByName<T>(IRepository<T> repository, string name) where T : Person // more specific constraint
        {
            return repository.GetAll().FirstOrDefault(p => p.Name == name);
        }
    }
}
```

**Explanation:**

- `Entity` is a base class with an `Id` property. The repository constraint `where T : Entity` ensures we can access `Id`.
- `IRepository<T>` defines the contract.
- `InMemoryRepository<T>` implements it using a `List<T>`.
- The `Add` method assigns a new ID, showing how generics allow us to work with any `T` that derives from `Entity`.
- In `Main`, we use the repository with `Person` entities.
- The static generic method `FindByName` demonstrates a generic method with a constraint that `T` must be `Person` (or derived). This shows how you can add type‑specific behavior.

This example highlights how generics enable reusable data access code while maintaining type safety.

---

## 13.7 Common Pitfalls and Best Practices

### 1. Name Type Parameters Clearly

Use descriptive names like `TKey`, `TValue`, `TEntity`, not just `T` when there are multiple parameters. This improves readability.

### 2. Apply Constraints Judiciously

Only add constraints that are necessary. Over‑constraining reduces reusability.

### 3. Prefer Generic Collections Over Non‑Generic Ones

Always use `List<T>`, `Dictionary<TKey, TValue>`, etc., instead of `ArrayList` or `Hashtable`. They are faster, type‑safe, and integrate with LINQ.

### 4. Be Aware of Type Inference

The compiler can often infer generic type arguments, making code cleaner. But if inference fails, you'll need to specify them explicitly.

### 5. Understand Covariance and Contravariance Limitations

Variance only works with reference types and only for interfaces/delegates. If you need variance with value types, you'll have to use workarounds (e.g., boxing).

### 6. Avoid Using Generic Types as Method Return Types Without Need

Returning `T` is fine, but returning a specific type like `List<T>` might limit future changes. Prefer interfaces like `IEnumerable<T>` or `IReadOnlyList<T>` where appropriate.

### 7. Use `default` for Default Values

When you need to return a default value for a generic type, use `default(T)` (or `default`). It handles both reference and value types correctly.

### 8. Be Cautious with Static Members in Generic Classes

Each closed generic type (e.g., `Stack<int>` and `Stack<string>`) has its own set of static members. This can be surprising.

```csharp
public class Counter<T>
{
    public static int Count;
}

Counter<int>.Count++;
Counter<string>.Count++;
Console.WriteLine(Counter<int>.Count); // 1
Console.WriteLine(Counter<string>.Count); // 1 (different)
```

### 9. Use Constraints to Enable Operations

If you need to use operators like `+` or compare with `<`, you'll need constraints that provide those capabilities (e.g., `IComparable<T>`). There's no constraint for operators, so you may need to use workarounds like dynamic or specialized interfaces.

### 10. Test with Different Types

When writing generic code, test it with at least one reference type and one value type to ensure it behaves correctly for both.

---

## 13.8 Chapter Summary

In this chapter, you've explored one of the most powerful features of C#: generics.

- **Generic classes** allow you to create type‑safe data structures without committing to a specific type.
- **Generic methods** enable algorithms that work with various types.
- **Constraints** (`where`) let you specify requirements for type parameters, enabling you to call methods or constructors on them.
- The **`default` keyword** provides a way to get the default value for any type.
- **Covariance (`out`) and contravariance (`in`)** make generic interfaces and delegates more flexible, allowing natural conversions between types in an inheritance hierarchy.
- A practical repository example showed how generics lead to reusable, type‑safe code.

Generics are everywhere in modern C# – from collections to LINQ to async patterns. Mastering them is essential for writing robust, reusable libraries and applications.

In the next chapter, **Exception Handling – Writing Robust Code**, you'll learn how to anticipate and handle errors gracefully. We'll cover try/catch/finally, creating custom exceptions, and best practices for building resilient applications.

**Exercises:**

1. Create a generic `Stack<T>` class with `Push`, `Pop`, and `Peek` methods. Add a constraint that `T` must be a reference type (just for practice). Test with `string` and a custom class.
2. Write a generic method `FindMax<T>` that takes an array of `T` and returns the maximum element, using `Comparer<T>.Default`. Ensure it works for `int`, `string`, and `DateTime`.
3. Implement a generic `Pair<TFirst, TSecond>` class with properties `First` and `Second`. Add a method `Swap` that returns a new pair with the values swapped.
4. Create an interface `IRepository<T>` and implement a `FileRepository<T>` that stores entities in a JSON file. Use a constraint to ensure `T` is serializable (hint: `where T : class` and use `System.Text.Json`).
5. Experiment with covariance: define a covariant interface `IProducer<out T>` and see how you can assign `IProducer<Derived>` to `IProducer<Base>`.

Now, get ready to handle errors gracefully in Chapter 14!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='12. delegates_events_and_lambda_expressions.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='14. exception_handling_writing_robust_code.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
